In [1]:
import spacy

nlp = spacy.load('en_core_web_md')




/usr/local/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation


In [3]:
import random

from math import cos
from math import sin


In [4]:
# answers to holidays
docs = [
    'I went to France to visit Paris',
    'New York City was crowded',
    'sightseeing in Bali',
    'swimming in the tropical oceans',
    'swimming in Bali',
    'raining in Paris',
    'visiting monuments in Paris',
    'San Francisco',
    'San Francisco',
    'flew to Berlin',
    'enjoyed visiting eiffel tower in paris',
    'Paris, city of love',
    'Flying is fun',
    'Paris is crowded',
    'France'
]

docs = [x.strip('\n') for x in open('answers.txt').readlines()]

In [5]:
# Remove stop words

def stop_word_stripper(line):
    stop_words = [w.strip('\n').lower() for w in open('stop_words.txt').readlines()]
    pos_stopper = ['PUNCT', 'SYM']
    return ' '.join([token.text for token in line if str(token).lower() not in stop_words and token.pos_  not in pos_stopper])

stripped_docs = [] #spacy object
copy_docs = [] # strings
for d in docs:
    stripped_docs.append(nlp(stop_word_stripper(nlp(d))))
    copy_docs.append(stop_word_stripper(nlp(d)))
    
print('stripped docs', stripped_docs)

stripped docs [become vegetarian, use electric cars, use public transport, Make big companies pollute less, use solar, clean energy less meat less burger shorter shower bath dry toilet electric car, reduce carbon emissions, overconsumption planned obsolescence, Pollute less Renewable energy Better industies, ecology renewable energy, Cleean energies, Clean energies, bike work, pollute less, use reusable items, Power home renewable energy, Invest energy efficient appliances, Reduce water waste, eat food buy make less meat, fuel efficient vehicle, nuclear fusion wind turbines solar pannels tidal turbines education, green ecology electric solar wind nuclear, Shrink carbon profile, Sensibilisation research, Replace regular incandescent light bulb, Drive less Carpool, Reduce Reuse Recycle, Go Solar, buy energy efficient appliances, Reduce waste, Avoid products packaging, Turn lights, Turn Electronic Devices, Plant Tree, Use Clean Fuel, education, education, teach, electric cars, Renewable F

In [6]:
# parse through to get entities 
kw_freq = {}

for i in range(len(stripped_docs)):
    line = stripped_docs[i]
    for e in line.ents:
        copy_docs[i] = copy_docs[i].replace(e.text, '').strip()
        if e.text in kw_freq:
            kw_freq[e.text] += 1
        else:
            kw_freq[e.text] = 1

print(kw_freq, copy_docs)

{'Tupperware': 1, 'Air Conditioning': 1} ['become vegetarian', 'use electric cars', 'use public transport', 'Make big companies pollute less', 'use solar', 'clean energy less meat less burger shorter shower bath dry toilet electric car', 'reduce carbon emissions', 'overconsumption planned obsolescence', 'Pollute less Renewable energy Better industies', 'ecology renewable energy', 'Cleean energies', 'Clean energies', 'bike work', 'pollute less', 'use reusable items', 'Power home renewable energy', 'Invest energy efficient appliances', 'Reduce water waste', 'eat food buy make less meat', 'fuel efficient vehicle', 'nuclear fusion wind turbines solar pannels tidal turbines education', 'green ecology electric solar wind nuclear', 'Shrink carbon profile', 'Sensibilisation research', 'Replace regular incandescent light bulb', 'Drive less Carpool', 'Reduce Reuse Recycle', 'Go Solar', 'buy energy efficient appliances', 'Reduce waste', 'Avoid products packaging', 'Turn lights', 'Turn Electronic 

In [7]:
# get lemma keywords 
# join the rest of the words together: 

corpus = nlp(' '.join(copy_docs))

MIN_CHARACTERS = 3

for token in corpus:
    if len(token.lemma_) > MIN_CHARACTERS:
        if token.lemma_ in kw_freq:
            kw_freq[token.lemma_] += 1
        else:
            kw_freq[token.lemma_] = 1

print(kw_freq)

{'Tupperware': 1, 'Air Conditioning': 1, 'become': 1, 'vegetarian': 1, 'electric': 4, 'public': 1, 'transport': 1, 'make': 2, 'company': 1, 'pollute': 3, 'less': 10, 'solar': 4, 'clean': 4, 'energy': 12, 'meat': 2, 'burger': 1, 'short': 1, 'shower': 1, 'bath': 1, 'toilet': 1, 'reduce': 7, 'carbon': 2, 'emission': 1, 'overconsumption': 1, 'plan': 1, 'obsolescence': 1, 'renewable': 6, 'better': 1, 'industie': 1, 'ecology': 2, 'cleean': 1, 'bike': 2, 'work': 1, 'reusable': 1, 'item': 1, 'power': 1, 'home': 1, 'invest': 1, 'efficient': 4, 'appliance': 2, 'water': 2, 'waste': 3, 'food': 1, 'fuel': 4, 'vehicle': 1, 'nuclear': 3, 'fusion': 2, 'wind': 2, 'turbine': 2, 'pannel': 1, 'tidal': 1, 'education': 3, 'green': 2, 'shrink': 1, 'profile': 1, 'sensibilisation': 1, 'research': 1, 'replace': 1, 'regular': 1, 'incandescent': 1, 'light': 2, 'bulb': 1, 'drive': 3, 'carpool': 1, 'reuse': 3, 'recycle': 2, 'avoid': 1, 'product': 2, 'packaging': 1, 'turn': 2, 'electronic': 1, 'devices': 1, 'plant':

In [8]:
# spellcheck keywords: 
from hunspell import Hunspell
h = Hunspell();
spell_checked_kw_freq = {}
for kw, freq in kw_freq.items():
    if freq == 1:
        found = False
        c_kws = h.suggest(kw) 
        for c in c_kws:
            if c in kw_freq:
                spell_checked_kw_freq[c] = kw_freq[c] + 1
                found = True
                break
        if not found:
            spell_checked_kw_freq[c_kws[0]] = 1
print(spell_checked_kw_freq) 

{'Tupperware': 2, 'Ear Conditioning': 1, 'become': 2, 'vegetarian': 2, 'public': 2, 'transport': 2, 'company': 2, 'burger': 2, 'Short': 1, 'shower': 2, 'bat': 1, 'toilet': 2, 'emission': 2, 'overconsumption': 2, 'plant': 4, 'obsolescence': 2, 'better': 2, 'industry': 1, 'clean': 5, 'wok': 1, 'reusable': 2, 'time': 1, 'powers': 1, 'hone': 1, 'incest': 1, 'food': 2, 'vehicle': 2, 'panel': 1, 'tidal': 2, 'shrunk': 1, 'profiles': 1, 'sensibility': 1, 'research': 2, 'replace': 2, 'regular': 2, 'incandescent': 2, 'bulb': 2, 'carpool': 2, 'avid': 1, 'packaging': 2, 'electronic': 2, 'devices': 2, 'teas': 1, 'hamburger': 2, 'ired': 1, 'rowel': 1, 'make': 3, 'lynch': 1, 'fossil': 2, 'conserve': 2, 'meat': 3, 'mart': 1}


In [9]:
# proper casing

caseless_freq = {}
propercase_freq = {}

for kw, count in kw_freq.items():
    if kw in caseless_freq:
        caseless_freq[kw.lower()].append(count)
    else:
        caseless_freq[kw.lower()] = [count]

for kw, count in kw_freq.items():
    if count == max(caseless_freq[kw.lower()]):
        propercase_freq[kw] = sum(caseless_freq[kw.lower()])

print(propercase_freq)
        

{'Tupperware': 1, 'Air Conditioning': 1, 'become': 1, 'vegetarian': 1, 'electric': 4, 'public': 1, 'transport': 1, 'make': 2, 'company': 1, 'pollute': 3, 'less': 10, 'solar': 4, 'clean': 4, 'energy': 12, 'meat': 2, 'burger': 1, 'short': 1, 'shower': 1, 'bath': 1, 'toilet': 1, 'reduce': 7, 'carbon': 2, 'emission': 1, 'overconsumption': 1, 'plan': 1, 'obsolescence': 1, 'renewable': 6, 'better': 1, 'industie': 1, 'ecology': 2, 'cleean': 1, 'bike': 2, 'work': 1, 'reusable': 1, 'item': 1, 'power': 1, 'home': 1, 'invest': 1, 'efficient': 4, 'appliance': 2, 'water': 2, 'waste': 3, 'food': 1, 'fuel': 4, 'vehicle': 1, 'nuclear': 3, 'fusion': 2, 'wind': 2, 'turbine': 2, 'pannel': 1, 'tidal': 1, 'education': 3, 'green': 2, 'shrink': 1, 'profile': 1, 'sensibilisation': 1, 'research': 1, 'replace': 1, 'regular': 1, 'incandescent': 1, 'light': 2, 'bulb': 1, 'drive': 3, 'carpool': 1, 'reuse': 3, 'recycle': 2, 'avoid': 1, 'product': 2, 'packaging': 1, 'turn': 2, 'electronic': 1, 'devices': 1, 'plant':

In [12]:
# semantic k means clustering

glove_vectors = []
labels_array = []

for kw, count in propercase_freq.items():
    labels_array.append(kw)
    glove_vectors.append(nlp(kw)[0].vector)

print(np.array(glove_vectors).shape, labels_array)

(84, 300) ['Tupperware', 'Air Conditioning', 'become', 'vegetarian', 'electric', 'public', 'transport', 'make', 'company', 'pollute', 'less', 'solar', 'clean', 'energy', 'meat', 'burger', 'short', 'shower', 'bath', 'toilet', 'reduce', 'carbon', 'emission', 'overconsumption', 'plan', 'obsolescence', 'renewable', 'better', 'industie', 'ecology', 'cleean', 'bike', 'work', 'reusable', 'item', 'power', 'home', 'invest', 'efficient', 'appliance', 'water', 'waste', 'food', 'fuel', 'vehicle', 'nuclear', 'fusion', 'wind', 'turbine', 'pannel', 'tidal', 'education', 'green', 'shrink', 'profile', 'sensibilisation', 'research', 'replace', 'regular', 'incandescent', 'light', 'bulb', 'drive', 'carpool', 'reuse', 'recycle', 'avoid', 'product', 'packaging', 'turn', 'electronic', 'devices', 'plant', 'tree', 'teach', 'hamburger', 'ride', 'towel', 'take', 'lunch', 'fossil', 'conserve', 'heat', 'smart']


In [13]:
# AffinityPropagation clustering 

AffinityPropagation_model = AffinityPropagation()
AffinityPropagation_model.fit(glove_vectors)

cluster_labels    = AffinityPropagation_model.labels_

clusters = {}
kw_cluster = {}
for i in range(len(labels_array)):
    if cluster_labels[i] not in clusters:
        clusters[cluster_labels[i]] = [labels_array[i]]
    else:
        clusters[cluster_labels[i]].append(labels_array[i])
    kw_cluster[labels_array[i]] = cluster_labels[i]

print (kw_cluster)

{'Tupperware': 5, 'Air Conditioning': 5, 'become': 0, 'vegetarian': 2, 'electric': 1, 'public': 0, 'transport': 5, 'make': 0, 'company': 5, 'pollute': 5, 'less': 0, 'solar': 1, 'clean': 0, 'energy': 1, 'meat': 2, 'burger': 2, 'short': 0, 'shower': 3, 'bath': 3, 'toilet': 3, 'reduce': 0, 'carbon': 5, 'emission': 5, 'overconsumption': 5, 'plan': 0, 'obsolescence': 5, 'renewable': 1, 'better': 0, 'industie': 5, 'ecology': 5, 'cleean': 5, 'bike': 4, 'work': 0, 'reusable': 7, 'item': 5, 'power': 1, 'home': 0, 'invest': 0, 'efficient': 0, 'appliance': 5, 'water': 0, 'waste': 7, 'food': 0, 'fuel': 1, 'vehicle': 5, 'nuclear': 1, 'fusion': 5, 'wind': 1, 'turbine': 5, 'pannel': 5, 'tidal': 5, 'education': 5, 'green': 5, 'shrink': 5, 'profile': 5, 'sensibilisation': 5, 'research': 5, 'replace': 0, 'regular': 0, 'incandescent': 6, 'light': 0, 'bulb': 6, 'drive': 0, 'carpool': 5, 'reuse': 7, 'recycle': 7, 'avoid': 0, 'product': 5, 'packaging': 5, 'turn': 0, 'electronic': 5, 'devices': 5, 'plant': 5

In [ ]:
'''
# k means clustering 

kmeans_model = KMeans(init='k-means++', n_clusters=4, n_init=5)
kmeans_model.fit(glove_vectors)


cluster_labels    = kmeans_model.labels_

clusters = {}
kw_cluster = {}
for i in range(len(labels_array)):
    if cluster_labels[i] not in clusters:
        clusters[cluster_labels[i]] = [labels_array[i]]
    else:
        clusters[cluster_labels[i]].append(labels_array[i])
    kw_cluster[labels_array[i]] = cluster_labels[i]

print (kw_cluster)
'''


In [14]:
#distance matrix (len(cluster_labels)^2)

from scipy import spatial

n = len(labels_array)

distance_matrix = np.zeros([n, n])

for i in range(n):
    for j in range(n):
        distance_matrix[i][j] = spatial.distance.cosine(glove_vectors[i], glove_vectors[j])


/usr/local/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: divide by zero encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/site-packages/scipy/spatial/distance.py:643: RuntimeWarning: overflow encountered in square
  vv = np.average(np.square(v), weights=w)
/usr/local/lib/python3.6/site-packages/scipy/spatial/distance.py:642: RuntimeWarning: overflow encountered in square
  uu = np.average(np.square(u), weights=w)
/usr/local/lib/python3.6/site-packages/scipy/spatial/distance.py:641: RuntimeWarning: overflow encountered in multiply
  uv = np.average(u * v, weights=w)
/usr/local/lib/python3.6/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [15]:
# assign max font size

def assign_font_size(propercase_freq, max_size, min_size):
    label_fs = {}
    sorted_tuples = [(k, propercase_freq[k]) for k in sorted(propercase_freq, key=propercase_freq.get, reverse=True)]
    min_count = sorted_tuples[-1][1]
    max_count = sorted_tuples[0][1]
    
    for kw, count in sorted_tuples:
        if (max_count - min_count) == 0:
            size = int((max_size - min_size) / 2.0 + min_size)
        else:
            #size = int(min_size + (max_size - min_size) * (count * 1.0 / (max_count - min_count)) ** 0.8)
            size = int((max_size - min_size)/(max_count - min_count)*count + min_size - (max_size - min_size)/(max_count - min_count)*min_count)
        label_fs[kw] = size
    
    return (label_fs)
        
kw_fs = assign_font_size(propercase_freq, 80, 30) #keyword_font_size
print(kw_fs)

{'energy': 80, 'less': 70, 'reduce': 57, 'renewable': 52, 'electric': 43, 'solar': 43, 'clean': 43, 'efficient': 43, 'fuel': 43, 'pollute': 39, 'waste': 39, 'nuclear': 39, 'education': 39, 'drive': 39, 'reuse': 39, 'plant': 39, 'tree': 39, 'make': 34, 'meat': 34, 'carbon': 34, 'ecology': 34, 'bike': 34, 'appliance': 34, 'water': 34, 'fusion': 34, 'wind': 34, 'turbine': 34, 'green': 34, 'light': 34, 'recycle': 34, 'product': 34, 'turn': 34, 'Tupperware': 30, 'Air Conditioning': 30, 'become': 30, 'vegetarian': 30, 'public': 30, 'transport': 30, 'company': 30, 'burger': 30, 'short': 30, 'shower': 30, 'bath': 30, 'toilet': 30, 'emission': 30, 'overconsumption': 30, 'plan': 30, 'obsolescence': 30, 'better': 30, 'industie': 30, 'cleean': 30, 'work': 30, 'reusable': 30, 'item': 30, 'power': 30, 'home': 30, 'invest': 30, 'food': 30, 'vehicle': 30, 'pannel': 30, 'tidal': 30, 'shrink': 30, 'profile': 30, 'sensibilisation': 30, 'research': 30, 'replace': 30, 'regular': 30, 'incandescent': 30, 'bu

In [16]:
def max_dimensions(kw_fs):
    kw_dimensions = {}
    for kw, fs in kw_fs.items():
        kw_dimensions[kw] = (int(0.7*len(kw)*fs), fs) #x, y (i.e. width, height)
    return kw_dimensions

kw_max_dim = max_dimensions(kw_fs)
print(kw_max_dim)

{'energy': (335, 80), 'less': (196, 70), 'reduce': (239, 57), 'renewable': (327, 52), 'electric': (240, 43), 'solar': (150, 43), 'clean': (150, 43), 'efficient': (270, 43), 'fuel': (120, 43), 'pollute': (191, 39), 'waste': (136, 39), 'nuclear': (191, 39), 'education': (245, 39), 'drive': (136, 39), 'reuse': (136, 39), 'plant': (136, 39), 'tree': (109, 39), 'make': (95, 34), 'meat': (95, 34), 'carbon': (142, 34), 'ecology': (166, 34), 'bike': (95, 34), 'appliance': (214, 34), 'water': (119, 34), 'fusion': (142, 34), 'wind': (95, 34), 'turbine': (166, 34), 'green': (119, 34), 'light': (119, 34), 'recycle': (166, 34), 'product': (166, 34), 'turn': (95, 34), 'Tupperware': (210, 30), 'Air Conditioning': (336, 30), 'become': (125, 30), 'vegetarian': (210, 30), 'public': (125, 30), 'transport': (189, 30), 'company': (146, 30), 'burger': (125, 30), 'short': (105, 30), 'shower': (125, 30), 'bath': (84, 30), 'toilet': (125, 30), 'emission': (168, 30), 'overconsumption': (315, 30), 'plan': (84, 3

In [17]:
# you need x,y width height for each rectangle (word)
# r1 x, y, width, height 
# p1--------
#  |        |
#  |        |
#  |--------p2

def rect_intersection(r1, r2): #return true if intersects
    p1 = {}
    p1["x"] = r1["x"]
    p1["y"] = r1["y"] - r1["height"]
    
    p2 = {}
    p2["x"] = r1["x"] + r1["width"]
    p2["y"] = r1["y"]
    
    p3 = {}
    p3["x"] = r2["x"]
    p3["y"] = r2["y"] - r2["height"]
    
    p4 = {}
    p4["x"] = r2["x"] + r2["width"]
    p4["y"] = r2["y"]
    
    return not(p2["y"] < p3["y"] or p1["y"] > p4["y"] or p2["x"] < p3["x"] or p1["x"] > p4["x"])

#test
'''
r1 = {
    "x" : 0,
    "y" : 10,
    "height" : 10,
    "width" : 10
}


r2 = {
    "x" : 5,
    "y" : 10,
    "height" : 10,
    "width" : 10
}

rect_intersection(r1, r2)
'''


'\nr1 = {\n    "x" : 0,\n    "y" : 10,\n    "height" : 10,\n    "width" : 10\n}\n\n\nr2 = {\n    "x" : 5,\n    "y" : 10,\n    "height" : 10,\n    "width" : 10\n}\n\nrect_intersection(r1, r2)\n'

In [18]:
class Word:
    def __init__(self, word, size, font_size, cluster):
        self.word = word
        self.width = size["width"] #{width, height}
        self.height = size["height"]
        self.font_size = font_size
        self.cluster = cluster

In [103]:
class Cloud:
    def __init__(self, words=[], canvas_size={"x": 1920, "y": 1080}, filename='clouds.html'):
        self.words = words
        self.canvas = [] #{word, font_size, x, y, width, height, color, cluster} <== color to be added
        self.canvas_size = canvas_size
        self.clusters = self.generate_clusters() # {0 : cluster0, 1 : cluster1, ...etc}
        self.filename = filename
        self.colors = ["#5F0B2B","#D11638", "#F08801", "#FACE00", "#ADA20B", "#D4D639", "#EF6D3E", "#90A6BF", "#F2AEB4", "#AA61AE", "#FBC6CE"]
        self.positions = []
        
    def generate_clusters(self):
        clusters = {}
        for w in self.words:
            if w.cluster in clusters:
                clusters[w.cluster].append(w)
            else:
                clusters[w.cluster] = [w]
        return clusters
    
    '''
    def choose_cluster_start(self):
        start_points = {}
        start_point = {}
        r = 0
        for i in range(len(self.clusters)):
            c = self.clusters[i]
            n = len(c)
            
            H = self.canvas_size["y"] #total height
            L = self.canvas_size["x"] #total length
            
            if i%2 == 0:
                y = random.randint(int(0.1*H), int(0.55*H))
            else:
                y = random.randint(int(0.55*H), int(0.9*H))
            x = random.randint(int(r*L), min(int((r+len(c)/len(self.words))*L), int(L*0.90)))
            
            r = min(0.85, r + len(c)/len(self.words))
            start_points[c[0].cluster] = {
                "x": x,
                "y": y
            }
        return start_points
    '''
        
    def create_cloud_svg(self):
        cl_size = {}
        for c, words in self.clusters.items():
            cl_size[c] = len(words)
        sorted_clusters = sorted(cl_size, key=cl_size.get)[::-1]
        
        start_position = { "x": 1920//2, "y": 1080//3 }
        
        for c in sorted_clusters:
            words = self.clusters[c]
            self.positions = self.spiral(start_position)
            for w in words:
                new_position = self.add_word_to_cloud(w) 
                
            start_position = new_position
            
        f = open(self.filename, 'w')
        f.write('<svg viewbox="0 0 1920 1080">')
        for w in cloud.canvas:
            f.write('<text x="{}" y="{}" font-family="Verdana" font-size="{}" stroke="none" fill="{}">'.format(w["x"], w["y"], w["font_size"], w["color"]))
            f.write(w["word"])
            f.write('</text>\n')
        f.write('</svg>')
        f.close()
        
        
    def add_word_to_cloud(self, word): # word class Word        
            
        
        for p in self.positions:
            if self.verify_overlap( word, p):
                self.positions.remove(p)
            else:
                self.canvas.append({
                    "word": word.word,
                    "x": p["x"],
                    "y": p["y"],
                    "width": word.width,
                    "height": word.height,
                    "font_size": word.font_size,
                    "color": self.colors[word.cluster],
                    "cluster": word.cluster
                })
                self.positions.remove(p)
                return p
            

    def rect_intersection(self, r1, r2):
        p1 = {}
        p1["x"] = r1["x"]
        p1["y"] = r1["y"] - r1["height"]

        p2 = {}
        p2["x"] = r1["x"] + r1["width"]
        p2["y"] = r1["y"]

        p3 = {}
        p3["x"] = r2["x"]
        p3["y"] = r2["y"] - r2["height"]

        p4 = {}
        p4["x"] = r2["x"] + r2["width"]
        p4["y"] = r2["y"]

        return not(p2["y"] < p3["y"] or p1["y"] > p4["y"] or p2["x"] < p3["x"] or p1["x"] > p4["x"])

    
    def verify_overlap(self, word, position): # true if overlaps, false if not
        new_rect = {
            "x": position["x"],
            "y": position["y"],
            "width": word.width,
            "height": word.height
        }
        for filled_rect in self.canvas:
            if self.rect_intersection(filled_rect, new_rect):
                return True
        #verify out of bound of rectangle:
        if new_rect["x"] < 0 or new_rect["x"] + new_rect["width"] > 1920 or new_rect["y"] > 1080 or new_rect["y"]- new_rect["height"] < 0:
            return True
        return False
    

    def spiral(self, start_point): # returns an [] with positions to test 
        points = [start_point]
        # x = (a + b*theta)cos(theta)
        # y = (a + b*theta)sin(theta)

        # b = a final - a ini / 2 pi n  n=number of turns
        a_ini = 0
        # a_final = self.canvas_size["x"]*len(self.clusters[cluster])/len(self.words) #spiral radius 
        a_final = self.canvas_size["x"] #spiral radius 

        b = (a_final - a_ini)/(2*3.14159*(self.canvas_size["y"]/10))

        thetas = [ (self.canvas_size["y"]/10 * 2)/1000 *x for x in range(1000)]
        for i in thetas: #1000 points
            x = ( a_ini + b*i )*cos(i) + start_point["x"]
            y = ( a_ini + b*i )*sin(i) + start_point["y"]
            points.append({"x": x, "y": y})

        return points
    
    '''
    def compress(self):
        # pull words towards the one zith the most occurence
        # create line 
        # test positions along that line 
        sizes = []
        for w in self.canvas:
            sizes.append(w["font_size"])
        central_word = self.canvas[sizes.index(max(sizes))]
        
        for w in self.canvas:
            if w["cluster"] != central_word["cluster"]:
                # sort tham by distance 
                pos_central_word = np.array([central_word["x"], central_word["y"]])
                pos_w = np.array([w["x"], w["y"]])
                dist = numpy.sqrt(numpy.sum((pos_central_word - pos_w)**2))
                
                # draw line 
                # inch closer 
                coeff = central_word["y"] - w["y"] / central_word["x"] - w["x"]
                coordiantes = [{"x": central_word["x"] + (central_word["x"] - w["x"])/100 * i, "y": central_word["y"] + coeff* (central_word["x"] - w["x"])/100 * i } for i in range(100)]
                for c in coordinates:
                    for word in self.words:
                        if self.verify_overlap(word, c):
                            break
    '''             

In [104]:
words = []
for kw, d in kw_max_dim.items():
    words.append(Word(kw, {"width": d[0], "height": d[1]}, kw_fs[kw], kw_cluster[kw]))

cloud = Cloud(words=words)

cloud.create_cloud_svg()

#cloud.compress()

#print(cloud.canvas)
'''
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(10,10))

ax = fig.add_subplot(111)
fig.subplots_adjust(top=0.85)

for w in cloud.canvas:
    ax.text(w["x"], w["y"], w["word"], fontsize=w["font_size"]//3)

ax.axis([0, 1920, 0, 1080])

'''
#plt.show()


'\nimport matplotlib.pyplot as plt\n\nfig = plt.figure(figsize=(10,10))\n\nax = fig.add_subplot(111)\nfig.subplots_adjust(top=0.85)\n\nfor w in cloud.canvas:\n    ax.text(w["x"], w["y"], w["word"], fontsize=w["font_size"]//3)\n\nax.axis([0, 1920, 0, 1080])\n\n'

In [ ]:
class Vertex:
    def __init__(self, node):
        self.id = node # we have a dict {id : { word, weight } }
        self.adjacent = {}

    def __str__(self):
        return str(self.id) + ' adjacent: ' + str([x.id for x in self.adjacent])

    def add_neighbor(self, neighbor, weight=0):
        self.adjacent[neighbor] = weight

    def get_connections(self):
        return self.adjacent.keys()  

    def get_id(self):
        return self.id

    def get_weight(self, neighbor):
        return self.adjacent[neighbor]


In [ ]:
class Graph:
    def __init__(self):
        self.vert_dict = {}
        self.num_vertices = 0

    def __iter__(self):
        return iter(self.vert_dict.values())

    def add_vertex(self, node):
        self.num_vertices = self.num_vertices + 1
        new_vertex = Vertex(node)
        self.vert_dict[node] = new_vertex
        return new_vertex

    def get_vertex(self, n):
        if n in self.vert_dict:
            return self.vert_dict[n]
        else:
            return None

    def add_edge(self, frm, to, cost = 0):
        if frm not in self.vert_dict:
            self.add_vertex(frm)
        if to not in self.vert_dict:
            self.add_vertex(to)

        self.vert_dict[frm].add_neighbor(self.vert_dict[to], cost)
        self.vert_dict[to].add_neighbor(self.vert_dict[frm], cost)

    def get_vertices(self):
        return self.vert_dict.keys()
    

In [ ]:
'''
Star Forest Clustering and putting together
'''

def StarForestAlgo(g):
    '''
    g = similarity graph
    '''
    stars = []
    while True:
        usedVertices = []
        bestCenter = findBestCenter(g, usedVertices)

        if bestCenter is None:
            break; 
        
        star, usedVertices = createGraphStar(g, bestCenter, usedVertices) # graph, vertice, [vertices]
        print(usedVertices)
        stars.append(star)
        
        
    return stars


def findBestCenter(g, usedVertices): # graph, [vertices]
    best_sum = 0
    best_center = None
    for v in g.get_vertices():
        if v not in usedVertices:
            sum = getSumOfConnectedEdges(g, v, usedVertices)
            if sum > best_sum:
                best_center = v
    return best_center


def getSumOfConnectedEdges(g, v, usedVertices):
    sum = 0
    connections = g.get_vertex(v).get_connections()
    for c in connections:
        if c not in usedVertices:
            sum += g.get_vertex(v).get_weight(c)
    return sum
    

def createGraphStar(g, bestCenter, usedVertices):
    star = Graph()
    for v in g.get_vertex(bestCenter).get_connections():
        if v not in usedVertices and g.get_vertex(bestCenter) != v:
            star.add_edge(bestCenter, v, g.get_vertex(bestCenter).get_weight(v))
            print(v)
            usedVertices.append(v)
    return star, usedVertices



g = Graph()

g.add_vertex('a')
g.add_vertex('b')
g.add_vertex('c')
g.add_vertex('d')
g.add_vertex('e')
g.add_vertex('f')

g.add_edge('a', 'b', 7)  
g.add_edge('a', 'c', 9)
g.add_edge('a', 'f', 14)
g.add_edge('b', 'c', 10)
g.add_edge('b', 'd', 15)
g.add_edge('c', 'd', 11)
g.add_edge('c', 'f', 2)
g.add_edge('d', 'e', 6)
g.add_edge('e', 'f', 9)

#StarForestAlgo(g)


In [ ]:
# Aspect ration of words : font_size (length) font_size*0.7(width)
# Aspect ration of SVG file is 16:9

# How to draw V1 
# Create a polygon with the number of vertices = number of clusters 
# here cluster size = 3 so a triangle (not ever going to exceed 5)
# 3 rectangles to fit within the first rectangle 

# in a 16:9

# Cluster one in rect 1 (y = 16, 9/4) (w: 8, l: 9/2)
# cluster Two in rect 2 (y = 16, 9/4*3) (w: 8, l: 9/2)
# Cluster three in rect 3 (y = 8, 9/4) Biggest cluster ? (w: 8, l: 9/2)

# Where to put the words 
# Start with the highest frequence with the biggest font : assign max font size before starting to draw
# If the next one is smaller in frequence, font size drops by 
# font size 35 to 18
# random choice where the word fits 

In [ ]:
'''
    def seam_carving(self):
        board = self.make_board()
        sparse = 0
        while sparse < 500:
            print(sparse)
            sparse += 1
            self.find_and_remove_path(board)
            board = self.make_board()
            
    def find_and_remove_path(self, board):
        v_path = []
        h_path = []
        sparse = True
        for y in range(self.canvas_size["y"]):
            pt = {}
            pt["y"] = y
            pt["x"] = np.argmin(board[y])
            v_path.append(pt) # first step 
            
        for x in range(self.canvas_size["x"]):
            pt = {}
            pt["x"] = x
            pt["y"] = np.argmin(board[:,x])
            h_path.append(pt) # first step 
        
        for p in v_path + h_path:
            for w in self.canvas:
                if w["x"] > p["x"]:
                    w["x"]-= 1
                if w["y"] > p["y"]:
                    w["y"] -= 1
                    
        board = self.make_board()
        return sparse

    
    def make_board(self):
        cv = self.canvas
        # map canvas to a 1920 1080 matrix 
        board = np.zeros(shape=(1080, 1920)) #lines, columns
        for w in cv:
            for i in range(int(w["y"]) - int(w["height"]), int(w["y"])+1):
                board[i][int(w["x"]) : int(w["x"]) + int(w["width"]) +1 ] = 1
        return board
'''